# Exercises on thermal policies
Combine both gem5-X, power model, and 3D-ICE to investigate and optimize the thermal behavior of the system. For such purpose, we providided a series of exercises for you to follow. For your convenience, we skip the time-consuming simulation process of gem5-X by provding a series of power traces of perlbench benchmark. You can find the power traces in the folder "data". The power traces are generated by running the perlbench benchmark in gem5-X for 9 different frequency levels. More specifically, they are 0.12720, 0.24020, 0.64980, 1.0045, 1.3174, 1.5973, 1.8505, 2.0817, 2.2943 GHz.

## EX1: Simple thermal policy with frequency scaling

In [ ]:
# Import necessary packages, and define the useful functions
import pandas as pd
import subprocess
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

path_3d_ice = "../../../WiPLASH/3d-ice/bin/3D-ICE-Emulator"
freq_level_list = [0.12720, 0.24020, 0.64980, 1.0045, 1.3174, 1.5973, 1.8505, 2.0817, 2.2943]

# The function to create the flp file for 3D-ICE
def create_flp_uniform(freq_level, flp_name):
    with open('data/freq_level'+str(freq_level)+'.csv', 'r') as f:
        content=f.readlines()
    with open(flp_name, 'w') as f:
        for i in range(4):
            block_name='core'+str(i)
            f.write("{0} :\n  position  {1},   {2} ;\n  dimension  {3},   {4} ;\n  power values  {5} ;\n\n".format(block_name, 1000*(i%2), 1000*int(i/2), 1000, 1000, content[0][0:-1]))

def create_flp(freq_level, flp_name, start_index):
    with open(flp_name, 'w') as fflp:
        for i in range(4):
            with open('data/freq_level'+str(freq_level[i])+'.csv', 'r') as fcsv:
                content=fcsv.readlines()
            block_name='core'+str(i+start_index)
            fflp.write("{0} :\n  position  {1},   {2} ;\n  dimension  {3},   {4} ;\n  power values  {5} ;\n\n".format(block_name, 1000*(i%2), 1000*int(i/2), 1000, 1000, content[0][0:-1]))
                   
def plot_cores(result_file):
    df = pd.read_csv(result_file, sep='\s\t\s', header=1, engine='python')
    fig, ax = plt.subplots()
    for i in range(len(df.columns)-1):
        df.plot(x='% Time(s)', y=df.columns.values[i+1],ax=ax)
    ax.legend(df.columns.values[1:])
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Temperature (K)")
    plt.hlines(340, 0, df.shape[0]/10, colors='k', linestyles='dashed')

def plot_2layer_cores(result_file_top, result_file_bottom):
    df1 = pd.read_csv(result_file_top, sep='\s\t\s', header=1, engine='python')
    df2 = pd.read_csv(result_file_bottom, sep='\s\t\s', header=1, engine='python')
    df2.pop(df2.columns[0])
    df = pd.concat([df1, df2], axis=1)
    df = pd.concat([df1, df2], axis=1)
    fig, ax = plt.subplots()
    for i in range(len(df.columns)-1):
        df.plot(x='% Time(s)', y=df.columns.values[i+1],ax=ax)
    ax.legend(df.columns.values[1:])
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Temperature (K)")
    plt.hlines(340, 0, df.shape[0]/10, colors='k', linestyles='dashed')


def parse_flp_file(filename):
    data = {}
    counter = 0
    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('position') or line.startswith('dimension'):
                key, x, y =line.replace(', ', ' ').split()[0:3]
                key=key+str(counter)
                data[key] = [float(x), float(y)]
                if line.startswith('dimension'):
                    counter += 1
            else:
                continue
    return data

def plot_rectangles(rectangles_dict):
    fig, ax = plt.subplots()
    ax.set_aspect('equal')

    for key in rectangles_dict:
        if key.startswith('position'):
            position = rectangles_dict[key]
            dimension_key = 'dimension' + key[-1]
            dimension = rectangles_dict[dimension_key]
            rect = Rectangle(position, dimension[0], dimension[1], alpha=0.5, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            ax.text(position[0]+dimension[0]/2, position[1]+dimension[1]/2, 'core'+key[-1],
                horizontalalignment='center',
                verticalalignment='center',)

    plt.xlim(0, max(rectangles_dict['position0'][0], rectangles_dict['position1'][0]) +
              rectangles_dict['dimension0'][0])
    plt.ylim(0, max(rectangles_dict['position2'][1], rectangles_dict['position3'][1]) +
              rectangles_dict['dimension2'][1])

    plt.xlabel('X (mm)')
    plt.ylabel('Y (mm)')
    plt.title('Floorplan')
    plt.show()

1. Run 3D-ICE with given power trace @ 2.2943 GHz


In [ ]:
freq_level = 9
create_flp_uniform(freq_level,'flp_demo.flp')
print("run at freq: ", freq_level_list[freq_level-1], "GHz")
run_output = subprocess.run([path_3d_ice, "stk_demo.stk"], stdout=subprocess.DEVNULL)

2. Plot the temperature trace and check if the temperature is within the safe range (340K)


In [ ]:
plot_cores('CPU_DIE_flp.txt')

3. Lower down the frequency and run 3D-ICE again


In [ ]:
freq_level = 8
create_flp_uniform(freq_level,'flp_demo.flp')
print("run at freq: ", freq_level_list[freq_level-1], "GHz")
run_output = subprocess.run([path_3d_ice, "stk_demo.stk"], stdout=subprocess.DEVNULL)

4. Plot the temperature trace and check if the temperature is within the safe range. Otherwise, go to step 3 and repeat the process until the temperature is within the safe range.

In [ ]:
plot_cores('CPU_DIE_flp.txt')

## EX2: Frequency scaling for each core


In the previous exercises, we lower down the frequency of all cores at the same time. They are common in some designs. However, it is possible to have different cores run at different frequency. In such case, we can lower down the frequency of the idle cores and keep the frequency of the active cores unchanged. In this exercise, we will implement such policy.

1. For instance, each core's frequency level can be specified in the follow example with a freq_level array, with four elements indicate four cores' frequency.

In [ ]:
freq_level = [9,9,9,9]
create_flp(freq_level,'flp_demo.flp',start_index=0)
print("run at freq: ", freq_level_list[freq_level[0]-1],freq_level_list[freq_level[1]-1],freq_level_list[freq_level[2]-1],freq_level_list[freq_level[3]-1], "GHz")
run_output = subprocess.run([path_3d_ice, "stk_demo.stk"], stdout=subprocess.DEVNULL)

2. Plot the temperature of the cores

In [ ]:
plot_cores('CPU_DIE_flp.txt')

3. Go to step 1 and change the frequency of each core until the temperature is within the safe range.

4. Propose a hypothesis on why the final frequency levels are chosen to maintain the temperature within the safe range. You can use the temperature trace and the core floorplan to support your hypothesis. The follow code is to draw the floorplan of the cores.

In [ ]:
flp_data=parse_flp_file('flp_demo.flp')
plot_rectangles(flp_data)

## EX3: Task mapping on 3D MPSoC

Now we will move to the 3D MPSoC. 3D MPSoC usually contains multilayer of cores. For instance, we can have 4 cores in each layer and 2 core layers in total.
In this exercise, we will investigate the thermal behavior of the 3D MPSoC by specifing the frequency level of each core. The frequency level of each core is specified in the follow example with a freq_level array, with eight elements indicate eight cores' frequency.

#1 Give the frequency level of each core and run 3D-ICE

In [ ]:
freq_level = [6,6,6,6,6,6,6,6]
create_flp(freq_level[0:4],'flp_top_demo.flp',start_index=0)
create_flp(freq_level[4:8],'flp_bot_demo.flp',start_index=4)
print("Top layer cores run at freq: ", freq_level_list[freq_level[0]-1],freq_level_list[freq_level[1]-1],freq_level_list[freq_level[2]-1],freq_level_list[freq_level[3]-1], "GHz")
print("Bottom layer cores run at freq: ", freq_level_list[freq_level[4]-1],freq_level_list[freq_level[5]-1],freq_level_list[freq_level[6]-1],freq_level_list[freq_level[7]-1], "GHz")
run_output = subprocess.run([path_3d_ice, "stk_3D_demo.stk"], stdout=subprocess.DEVNULL)

#2 Plot the temperature of the cores

In [ ]:
plot_2layer_cores('CPU_DIE_TOP_flp.txt', 'CPU_DIE_BOT_flp.txt')

#3 Check the differences of each core's temperature and explain why the temperature is different even the frequency/power of each core is the same.

#4 Go back to step 1 and think about the task mapping strategy to activate 6 active cores on the target MPSoC. You can use the frequency level 1 to deactivate the core and use the frequency level larger than 1 to activate the core.